In [78]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import logging
import pandas as pd
import numpy as np
# sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append('../')

from abacus.splitter.params import SplitBuilderParams
from abacus.splitter.split_builder import SplitBuilder

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Кейс №6. Выделение групп

In [79]:
df = pd.read_csv('./data/ab_data.csv', nrows=5000)
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["country"] = np.random.randint(1, 3, df.shape[0])
df["id"] = df.index
print(df.head())

   height_now  height_prev  weight_now  weight_prev  noise_now  noise_prev  \
0  172.470442   166.382868  163.648256   173.422115   5.644384   -1.923929   
1  178.610565   173.386388  173.195935   180.831845  11.664751    2.711829   
2  180.382301   178.563579  174.520065   181.571024   7.183178   -0.453939   
3  180.954018   179.732080  175.731245   182.840388  -4.682656    6.358723   
4  167.590042   167.684244  166.109779   170.189261   9.134517   -1.153977   

  groups  id moda_city  country  numerator  denominator  conversion  
0      A   0         3        1          3            2           1  
1      B   1         4        2          1            4           1  
2      A   2         2        2          2            2           1  
3      B   3         4        2          3            2           0  
4      B   4         1        1          3            2           1  


In [80]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': None,
        'target': None
    },
    main_strata_col = "moda_city",
    split_metric_col = "height_now",
    id_col = "id",
    cols = ["height_prev"],
    cat_cols=["country"],
    pvalue=0.05,
    n_bins = 6,
    min_cluster_size = 500
)

In [81]:
split_builder = SplitBuilder(df, split_builder_params)

In [82]:
split = split_builder.collect()

<class 'abacus.auto_ab.params.DataParams'>
DataParams(id_col='id', group_col='group_name', control_name='control', treatment_name='target', strata_col='country', target='height_now', target_flg='bought', predictors=['weight_now'], numerator='clicks', denominator='sessions', covariate='height_prev', target_prev='height_prev', predictors_prev=['weight_prev'], is_grouped=True, control=array([], dtype=float64), treatment=array([], dtype=float64))
<class 'abacus.auto_ab.params.HypothesisParams'>
HypothesisParams(alpha=0.05, beta=0.2, alternative='two-sided', split_ratios=(0.5, 0.5), strata='country', strata_weights={1: 0.8, 2: 0.2}, metric_type='solid', metric_name='mean', metric=<function mean at 0x7fa2a62a4c20>, n_boot_samples=200, n_buckets=50)
!!!
<class 'abacus.auto_ab.params.DataParams'>
DataParams(id_col='id', group_col='group_name', control_name='control', treatment_name='target', strata_col='country', target='height_prev', target_flg='bought', predictors=['weight_now'], numerator='

In [83]:
split.head()

,height_now,height_prev,weight_now,weight_prev,noise_now,noise_prev,groups,id,moda_city,country,numerator,denominator,conversion,strata,group_name
0,172.470442,166.382868,163.648256,173.422115,5.644384,-1.923929,A,0,3,0.494807,3,2,1,31-1,control
1,178.610565,173.386388,173.195935,180.831845,11.664751,2.711829,B,1,4,0.505216,1,4,1,44-1,target
2,180.382301,178.563579,174.520065,181.571024,7.183178,-0.453939,A,2,2,0.505216,2,2,1,25-1,target
3,180.954018,179.732080,175.731245,182.840388,-4.682656,6.358723,B,3,4,0.505216,3,2,0,45-1,target
4,167.590042,167.684244,166.109779,170.189261,9.134517,-1.153977,B,4,1,0.494807,3,2,1,10-1,target
